### Initial Setup

Installation of python packages and external applications

In [1]:
# Install scrubber
_ = !git clone --single-branch --branch develop https://github.com/forlilab/scrubber.git
_ = !cd scrubber; pip install --use-pep517 -e .; cd ..

In [2]:
# Install autodock Vina
_ = !apt install autodock-vina

In [3]:
# Install python modules
_ = !pip install  py3Dmol prody openbabel-wheel rdkit pandas meeko

In [1]:
# Insrtall pdb2pqr (maybe required to build missing atoms in receptor)
_  = !pip install pdb2pqr

In [2]:
#@title
def view_mol(molecule, style='stick', surface=False, opacity=0.7, surf_type='MS'):
    viewer = py3Dmol.view()
    viewer.addModel(open(molecule).read(), molecule.split('.')[-1])
    # Style the molecule (e.g., stick representation)
    viewer.setStyle({f'{style}': {'colorscheme': 'greenCarbon'}})

    if surface:
        viewer.addSurface(f'{surf_type}', {'opacity': opacity , 'color': 'white'})
    # Zoom to fit
    viewer.zoomTo()

    # Display in Jupyter Notebook or output HTML
    viewer.show()

### Import libraries

In [3]:
from openbabel import pybel # openbabel format conversion tool
from openbabel import openbabel as ob
from prody import parsePDB, writePDB  # for PDB processing
import py3Dmol  # Embedded molecular visualization on the notebook
import requests  # python library to process requests for Web sites

### Collect PDB files

In [11]:
# Retrieve the PDB files from the Protein Databank site
_ = !wget https://files.rcsb.org/download/1IEP.pdb  # BCR-ABL (Imatinib complex)
#_ = !wget https://files.rcsb.org/download/1M17.pdb  # EGFR WT (Erlotinib complex)
#_ = !wget https://files.rcsb.org/download/2JIT.pdb  # EGFR T790M mutant

### Ligand Preparation (Imatinib)

There are multiple ways in which small-molecules ligands can be prepared for docking. For the first ligand, we will do the following:

1. Retrive the 3D `.sdf` structure file from the PubChem moluecar database
2. Use the `mek_prepare_ligand.py` to generate the `pdbqt` file used by Vina

NOTE: some input sdf files can cause errores and may require additional fixing. This will have to be dealt with on a case by case baseis.

In [5]:
# Retrivce Imatinib from PubChem (https://pubchem.ncbi.nlm.nih.gov/)
# Imatinib’s PubChem CID: 5291 (you would search PubChem by name to get this)

cid = "5291"
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/{cid}/record/SDF/?record_type=3d&response_type=save&response_basename=imatinib_3d"

# Download SDF
response = requests.get(url)
with open("imatinib.sdf", "wb") as f:
    f.write(response.content)

print("Downloaded Imatinib 3D SDF from PubChem!")

Downloaded Imatinib 3D SDF from PubChem!


Let's visualize the .sdf file using Py3Dmol

In [6]:
view_mol('imatinib.sdf', 'stick')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Wd should check the above strocture fro a few things>

1. Does it containt explicit hydrogens ? (both polar and non-polar?)
2. Does it look like a true 3D structure ? (We don't want a 2D sdf file with all atoms flat on one plane
3. Do the valences look right ?


Everything seems OK, so let's convert the molecle into `.pddbt`

**pdbqt** stands for "pdb with chargers (**q**) and torsions(**t**)"

In [7]:
!mk_prepare_ligand.py -i imatinib.sdf

Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0


Here we also wnant to visualize this file, but in this case we need to convert it first to `mol2` because Py3Dmol can read pdbqt files. We use `openbabel` to make the conversion:

In [8]:
!obabel -i pdbqt imatinib.pdbqt -o mol2 -O imatinib_prep.mol2 -h
view_mol('imatinib_prep.mol2')

1 molecule converted


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The above visualization should be very similar to the previous one, with an important difference: only the *polar hydrogens* should remain, all non-polar hydrogens must have been removed as needed by the Vina.

Let's view the contentes of the PDBQT file using the Linux `cat` command. The TORSDOF line at the end indicartes that the number of active bond rotaions in 7. These are use for changing the conformation of the molecule during docking.

In [9]:
!cat imatinib.pdbqt

REMARK SMILES Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc(-c2cccnc2)n1
REMARK SMILES IDX 8 1 6 2 7 3 9 5 10 6 22 7 11 8 21 9 12 10 13 11 14 12 15 13
REMARK SMILES IDX 20 14 16 15 19 16 17 17 18 18 5 19 23 20 4 21 24 22 3 23
REMARK SMILES IDX 2 24 1 25 25 26 26 28 37 29 27 30 30 31 28 32 29 33 31 34
REMARK SMILES IDX 32 35 36 36 33 37 35 38 34 39
REMARK H PARENT 6 4 25 27
ROOT
ATOM      1  O   UNL     1      -0.828   0.249  -1.042  1.00  0.00    -0.269 OA
ATOM      2  N   UNL     1      -0.107  -1.833  -0.233  1.00  0.00    -0.322 N 
ATOM      3  C   UNL     1       0.083  -0.535  -0.730  1.00  0.00     0.255 C 
ATOM      4  H   UNL     1       0.715  -2.371   0.029  1.00  0.00     0.170 HD
ENDROOT
BRANCH   3   5
ATOM      5  C   UNL     1       1.465  -0.118  -0.905  1.00  0.00     0.040 A 
ATOM      6  C   UNL     1       2.457  -1.071  -1.139  1.00  0.00     0.014 A 
ATOM      7  C   UNL     1       1.800   1.234  -0.836  1.00  0.00     0.014 A 
ATOM      8  C   UNL     1       3.

### Preparing the receptor protein

Opem the 1IEP file in PyMOL. We need to clean the structure by:

1. Remove duplicate molecules (keep only the B chain)
2. Remove solvent molecules (the water molecules are not used in this docking protocol)
3. Remove metal ions (the effect of metal ions upon ligand binding will not be considered)
4. Remove organic molecules including ligands (the 1IEP already containts an imatinib molecule boud at the active site)

In [12]:
#@title PDB file header:
!cat 1IEP.pdb |head -10

HEADER    TRANSFERASE                             10-APR-01   1IEP              
TITLE     CRYSTAL STRUCTURE OF THE C-ABL KINASE DOMAIN IN COMPLEX WITH STI-571. 
COMPND    MOL_ID: 1;                                                            
COMPND   2 MOLECULE: PROTO-ONCOGENE TYROSINE-PROTEIN KINASE ABL;                
COMPND   3 CHAIN: A, B;                                                         
COMPND   4 FRAGMENT: KINASE DOMAIN;                                             
COMPND   5 SYNONYM: P150, C-ABL;                                                
COMPND   6 EC: 2.7.1.112;                                                       
COMPND   7 ENGINEERED: YES                                                      
SOURCE    MOL_ID: 1;                                                            


In [13]:
#@title Use py3Dmol to visualize the 1IEP full unprocessed structure
# View 1IEP pdb file
import py3Dmol

# Load your structure (PDB file or ID)
view = py3Dmol.view(query='pdb:1iep')  # Example with STI-571 in 1T46
view.setStyle({'chain': 'A'},{'cartoon': {'color': 'teal'}})  # Color protein
view.setStyle({'chain': 'B'},{'cartoon': {'color': 'orange'}})  # Color protein

# Color ligand STI differently
view.addStyle({'resn': 'STI'}, {'stick': {'colorscheme': 'yellowCarbon', 'radius': 0.2}})
#view.zoomTo({'chain': 'A', 'resn': 'STI'})


view.addStyle({'resn': 'HOH'},  # HOH is the standard residue name for water
             {'stick': {'color': 'blue', 'radius': 0.15}})

view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The ProDy library offers a programmatic alternative the manual preparation of PDB files with PyMOL :

In [14]:
from prody import parsePDB, writePDB
struct = parsePDB("1IEP") # Read pdb file straight from the Protain Databank Site
view_mol('1IEP.pdb')  # visualize the receptor

@> Connecting wwPDB FTP server RCSB PDB (USA).
DEBUG:.prody:Connecting wwPDB FTP server RCSB PDB (USA).
@> Downloading PDB files via FTP failed, trying HTTP.
INFO:.prody:Downloading PDB files via FTP failed, trying HTTP.
@> 1iep downloaded (1iep.pdb.gz)
DEBUG:.prody:1iep downloaded (1iep.pdb.gz)
@> PDB download via HTTP completed (1 downloaded, 0 failed).
DEBUG:.prody:PDB download via HTTP completed (1 downloaded, 0 failed).
@> 4710 atoms and 1 coordinate set(s) were parsed in 0.06s.
DEBUG:.prody:4710 atoms and 1 coordinate set(s) were parsed in 0.06s.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The above image shows that the `1IEP.pdb` containts two copies of the ABL kinase in complex with imatinig (ligand with code `STI`). We need remove one of the two copies, as well as the copies of the imatinib molecule. Prody can do this:

In [15]:
receptor = struct.select("protein and chain B")  # Keep only protein atoms from the B chain
writePDB("1iep_clean_B.pdb", receptor)  # write the cleaned pdb file to disk
view_mol('1iep_clean_B.pdb')  # visualize the receptor

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

As you see above, the clean structure contains just one copy of the protein and no ligand.

Now, we use Meeko again to prepare the receptor:

1. Add hydrogens (but only in polar groups)
2. Add charges (not really necessary for docking with Vina)
3. Generate a file .pdbqt format for Vina docking

In [16]:
# generate the .pdbqt receptor file
!mk_prepare_receptor.py --read_pdb 1iep_clean_B.pdb -o 1iep_B -p

# Convert it to mol2 format for viewing with py3Dmol
!obabel -i pdbqt 1iep_B.pdbqt -o mol2 -O 1iep_B.mol2
view_mol('1iep_B.mol2')


Files written:
1iep_B.pdbqt <-- static (i.e., rigid) receptor input file
1 molecule converted


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In the model above, noctice that now all aminoacid residues have poloar hydrogens (non-polar hydrogens are removed). These polar hydrogen positions are important for computation of hydrogen bonds during the docking process.

Let's use the **Webina** tool to prepare the box of ligand for the docking run. got to https://durrantlab.pitt.edu/webina/ and upload the receptor and ligand structures

In Webina, we need to position the box inside which conformations will be searched. Go back to Pymol and find a residue close to the ligand.

No copy the values of the box center and position to the following cell, and run Vina. The program will produce a file name 'imatinib_docked' with extension `.pdbqt`. Note the parameter `exhaustiveness`set to 1 - this is the lowest value we can use here, and means that Vina do the smallest possible amount of energy surface search. It's very fast, but changes of finding the correct pose are small. Try with diffeent values of this parameter and observer the energy values.

In [27]:
mol = 'imatinib'  # the small-molecule ligand
receptor = '1iep_B.pdbqt'  # the protein receptor (chain B of 1IEP)
comment = ''
box_center_x = 13.171
box_center_y = 95.113
box_center_z = 58.594
box_size_x = 28.371
box_size_y = 34.47
box_size_z = 35.960
# seed = 734633875 # for exhaustiveness 1
seed = 0
exhaustiveness = 1 # very limited enerty search, increase for better results
out = f'{mol}_docked_e{exhaustiveness}{comment}.pdbqt'

!vina \
--receptor {receptor} \
--out {out} \
--ligand {mol}.pdbqt \
--exhaustiveness {exhaustiveness} \
--energy_range 100 \
--num_modes 10 \
--center_x {box_center_x} \
--center_y {box_center_y} \
--center_z {box_center_z} \
--size_x {box_size_x} \
--size_y {box_size_y} \
--size_z {box_size_z} \
--seed {seed}


AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

After the run is finished, a table displays the *docking poses* that were found by Vina, ranked in order of increasing energy (lowest energy means a more stable conformation)

The coordinates for the docking poses and respective energies are stored in a `.pdbqt` file, which in this case will be named  `imatinib_docked_e1.pdbqt` (the number after the "e" indicates the exhaustiveness level used in the run).

Using the folder option on the left pane, find the the `imatinib_docked_e1.pdbqt` and download to your computer, and then open it in `PyMOL` together with the `1IEP.pdb`file. Compare the docked poses with the experimental pose.

Repeat the above docking run, but this time with a higher exhaustiveness and compare the obtained energies (a more negative value is better) and docked conformations (compare the conformations with the experimental ligand conformation).

In [22]:
!cat imatinib_docked_e1.pdbqt |head -20

MODEL 1
REMARK VINA RESULT:   -12.953      0.000      0.000
REMARK INTER + INTRA:         -18.746
REMARK INTER:                 -18.189
REMARK INTRA:                  -0.558
REMARK UNBOUND:                -0.493
REMARK SMILES Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc(-c2cccnc2)n1
REMARK SMILES IDX 8 1 6 2 7 3 9 5 10 6 22 7 11 8 21 9 12 10 13 11 14 12 15 13
REMARK SMILES IDX 20 14 16 15 19 16 17 17 18 18 5 19 23 20 4 21 24 22 3 23
REMARK SMILES IDX 2 24 1 25 25 26 26 28 37 29 27 30 30 31 28 32 29 33 31 34
REMARK SMILES IDX 32 35 36 36 33 37 35 38 34 39
REMARK H PARENT 6 4 25 27
ROOT
ATOM      1  O   UNL     1      13.188  96.367  59.655  1.00  0.00    -0.269 OA
ATOM      2  N   UNL     1      11.726  97.750  60.862  1.00  0.00    -0.322 N 
ATOM      3  C   UNL     1      12.509  96.603  60.668  1.00  0.00     0.255 C 
ATOM      4  H   UNL     1      11.244  97.857  61.751  1.00  0.00     0.170 HD
ENDROOT
BRANCH   3   5
ATOM      5  C   UNL     1      12.480  95.625  61.743  1.00  0

#### Alternative way to generate the dockikng box

It is possible to generate the box automatically based on the expected position of the ligand (in our case, the experimental structure of the ligand).

First with extract the pdb structure of the ligand with prody, and then we use `mk_prepare_receptor.py` to create a box dimension file and box pdb file.

In [28]:
struct = parsePDB("1IEP") # Read pdb file straight from the Protain Databank Site
ligand = struct.select("resname STI and chain B")  # Extract the ligand molecule
writePDB("imatinib_exper.pdb", ligand)  # write the ligand to a PDB file
view_mol("imatinib_exper.pdb")

@> Connecting wwPDB FTP server RCSB PDB (USA).
DEBUG:.prody:Connecting wwPDB FTP server RCSB PDB (USA).
@> Downloading PDB files via FTP failed, trying HTTP.
INFO:.prody:Downloading PDB files via FTP failed, trying HTTP.
@> 1iep downloaded (1iep.pdb.gz)
DEBUG:.prody:1iep downloaded (1iep.pdb.gz)
@> PDB download via HTTP completed (1 downloaded, 0 failed).
DEBUG:.prody:PDB download via HTTP completed (1 downloaded, 0 failed).
@> 4710 atoms and 1 coordinate set(s) were parsed in 0.09s.
DEBUG:.prody:4710 atoms and 1 coordinate set(s) were parsed in 0.09s.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [33]:
!mk_prepare_receptor.py --read_pdb 1iep_clean_B.pdb -o temp_1iep -p -v \
--box_enveloping imatinib_exper.pdb --padding 5


Files written:
  temp_1iep.pdbqt <-- static (i.e., rigid) receptor input file
temp_1iep.box.txt <-- Vina-style box dimension file
temp_1iep.box.pdb <-- PDB file to visualize the grid box


In [34]:
!cat temp_1iep.box.txt

center_x = 13.171
center_y = 95.113
center_z = 58.594
size_x = 18.371
size_y = 24.476
size_z = 25.960

Visualiae the docking with with py3Dmol

In [35]:
view = py3Dmol.view(width=600, height=400)

# Load both PDB files into the same viewer
view.addModel(open("1iep_clean_B.pdb").read(), "pdb")
view.addModel(open("temp_1iep.box.pdb").read(), "pdb")
view.addModel(open("imatinib_exper.pdb").read(), "pdb")


# Style them differently (e.g., colors)
view.setStyle({"model": 0}, {"stick": {"color": "yellow"}})   # PDB 1 (red)
view.setStyle({"model": 1}, {"stick": {"color": "blue"}})  # PDB 2 (blue)
view.setStyle({"model": 2}, {"stick": {"color": "green"}})  # PDB 3 (blue)

# Zoom to fit both
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [37]:
!cat imatinib_exper.pdb

REMARK Selection 'resname STI and chain B'
HETATM    1  C1  STI B 202      16.356 100.406  50.614  1.00 54.02           C  
HETATM    2  C6  STI B 202      15.218 100.012  51.414  1.00 52.26           C  
HETATM    3  C5  STI B 202      15.089 100.453  52.798  1.00 49.43           C  
HETATM    4  C4  STI B 202      16.138 101.298  53.339  1.00 47.99           C  
HETATM    5  N3  STI B 202      17.220 101.660  52.543  1.00 51.14           N  
HETATM    6  C2  STI B 202      17.356 101.243  51.215  1.00 52.94           C  
HETATM    7  C7  STI B 202      13.928 100.078  53.673  1.00 48.61           C  
HETATM    8  C12 STI B 202      12.723  99.470  53.173  1.00 45.68           C  
HETATM    9  C11 STI B 202      11.704  99.172  54.114  1.00 47.75           C  
HETATM   10  N10 STI B 202      11.881  99.464  55.456  1.00 45.18           N  
HETATM   11  C9  STI B 202      13.053 100.050  55.907  1.00 48.27           C  
HETATM   12  N8  STI B 202      14.046 100.345  55.023  1.00 42.11

#### Alternative way to generate ligand structures

Now we wanto to create another molecules to dock into the ABL1 kinase structure. We can retrieve them from PubChem (or another database) as we did before, or can generate those molecules from SMILES strings, using the `scrub`tool. In this case we will build a structure for the ABL1 inhibitor `Ponatinib` using its SMILES string as a starting point.

In [ ]:
# Ponatinib
smiles_string="Cc1ccc(cc1C#Cc2cnc3n2nccc3)C(=O)Nc4ccc(c(c4)C(F)(F)F)CN5CCN(CC5)C"
!scrub.py "{smiles_string}" -o ponatinib.sdf --skip_tautomers

Scrub completed.
Summary of what happened:
Input molecules supplied: 1
mols processed: 1, skipped by rdkit: 0, failed: 0
nr isomers (tautomers and acid/base conjugates): 1 (avg. 1.000 per mol)
nr conformers:  1 (avg. 1.000 per isomer, 1.000 per mol)


In [ ]:
view_mol('ponatinib.sdf')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
!mk_prepare_ligand.py -i ponatinib.sdf
!obabel -i pdbqt ponatinib.pdbqt -o mol2 -O ponatinib_prep.mol2 -h
view_mol('ponatinib_prep.mol2')

Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0
1 molecule converted


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

We could have also retrieved ponatinib from PubChem

In [ ]:
# get ponatinib
# Nilotinib PubChem CID: 644241

mol_name = "nilotinib"
cid = "644241"
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/{cid}/record/SDF/?record_type=3d&response_type=save&response_basename=imatinib_3d"

# Download SDF
response = requests.get(url)
with open(f"{mol_name}.sdf", "wb") as f:
    f.write(response.content)

print(f"Downloaded {mol_name} 3D SDF from PubChem!")

Downloaded nilotinib 3D SDF from PubChem!


Now that we have the `.sdf` structure file from ponatinib, we can prepare it in the same way using `mk_prepare_ligand.py` .

----
**TO DO:** prepare the ponatinib molecule and dock it on the ABL1 active using Vina. Compare the results with those obtained with imatinib. Which one is the most potent ligand ?
